In [25]:
! cp /content/wavenet_model.py .
! cp /content/wavenet_modules.py .
! cp /content/audio_data.py .
! cp /content/wavenet_training.py .
! cp /content/model_logging.py .

cp: '/content/wavenet_model.py' and './wavenet_model.py' are the same file
cp: '/content/wavenet_modules.py' and './wavenet_modules.py' are the same file
cp: '/content/audio_data.py' and './audio_data.py' are the same file
cp: '/content/wavenet_training.py' and './wavenet_training.py' are the same file
cp: '/content/model_logging.py' and './model_logging.py' are the same file


In [26]:
import torch
from wavenet_model import *
from audio_data import WavenetDataset
from wavenet_training import *
from model_logging import *

In [27]:
# initialize cuda option
dtype = torch.FloatTensor # data type
ltype = torch.LongTensor # label type

use_cuda = torch.cuda.is_available()
if use_cuda:
    print('use gpu')
    dtype = torch.cuda.FloatTensor
    ltype = torch.cuda.LongTensor

In [28]:
model = WaveNetModel(layers=10,
                     blocks=3,
                     dilation_channels=32,
                     residual_channels=32,
                     skip_channels=1024,
                     end_channels=512, 
                     output_length=16,
                     dtype=dtype, 
                     bias=True)
# model = load_latest_model_from('snapshots', use_cuda=use_cuda)

print('model: ', model)
print('receptive field: ', model.receptive_field)
print('parameter count: ', model.parameter_count())

model:  WaveNetModel(
  (filter_convs): ModuleList(
    (0): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (1): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (2): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (3): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (4): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (5): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (6): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (7): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (8): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (9): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (10): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (11): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (12): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (13): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (14): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (15): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (16): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (1

In [29]:
data = WavenetDataset(dataset_file='/content/dataset.npz',
                      item_length=model.receptive_field + model.output_length - 1,
                      target_length=model.output_length,
                      file_location='/content/',
                      test_stride=500)
print('the dataset has ' + str(len(data)) + ' items')

one hot input
the dataset has 598277 items


In [30]:
def generate_and_log_samples(step):
    sample_length=32000
    gen_model = load_latest_model_from('snapshots', use_cuda=False)
    print("start generating...")
    samples = generate_audio(gen_model,
                             length=sample_length,
                             temperatures=[0.5])
    tf_samples = tf.convert_to_tensor(samples, dtype=tf.float32)
    logger.audio_summary('temperature_0.5', tf_samples, step, sr=16000)

    samples = generate_audio(gen_model,
                             length=sample_length,
                             temperatures=[1.])
    tf_samples = tf.convert_to_tensor(samples, dtype=tf.float32)
    logger.audio_summary('temperature_1.0', tf_samples, step, sr=16000)
    print("audio clips generated")


# logger = TensorboardLogger(log_interval=200,
#                            validation_interval=400,
#                            generate_interval=1000,
#                            generate_function=generate_and_log_samples,
#                            log_dir="logs/chaconne_model")

logger = Logger(log_interval=200,
                validation_interval=400,
                generate_interval=1000)

In [ ]:
trainer = WavenetTrainer(model=model,
                         dataset=data,
                         lr=0.001,
                         snapshot_path='snapshots',
                         snapshot_name='chaconne_model',
                         snapshot_interval=1000,
                         logger=logger,
                         dtype=dtype,
                         ltype=ltype)

print('start training...')
trainer.train(batch_size=16, epochs=10)

In [ ]:
start_data = data[250000][0] # use start data from the data set
start_data = torch.max(start_data, 0)[1] # convert one hot vectors to integers

def prog_callback(step, total_steps):
    print(str(100 * step // total_steps) + "% generated")

generated = model.generate_fast(num_samples=160000,
                                 first_samples=start_data,
                                 progress_callback=prog_callback,
                                 progress_interval=1000,
                                 temperature=1.0,
                                 regularize=0.)

In [ ]:
import IPython.display as ipd

ipd.Audio(generated, rate=16000)